## Sacred for Repeat Ecperiments
- [Sacred Github](https://github.com/IDSIA/sacred)
- Sacred?
  - Sacred is a tool to help configure, organize, log and reproduce experiments developed at IDSIA
  - Tools to help save and manage settings as we proceed with machine learning modeling
  - Why do we need it?
    - Frequent examples in Kaggle
      - What's the feature  used?
      - What parameters did you use?
      - What's the result?
    - We need tools to help us run various experiments quickly and automatically record them without having to record them by hand.
  - What if we implement it with Scratch?
    - Define Logger and store features, parameters, etc. in Logger whenever necessary.
    - Parsing Logger to view values
    - Sacred helps us easily use the above method as a decorator.

- Main mechanisms of Sacred
  - ConfigScopes : Can conveniently handle the function's local variable @ex.config decorator
  - Config Injection: All functions have access to settings
  - Command-line interface : Can be executed by changing parameters with a command line
  - Observer: All information from the experiment is provided to Observer and stored. MongoDB / S3 etc. => Just save it locally in this project.
  - Automatic seeding: Helps control randomness in an experiment

In [1]:
!pip3 install sacred

### Example of Sacred on the official website

In [2]:
from IPython.display import set_matplotlib_formats

set_matplotlib_formats('retina')

/var/folders/pw/83spn3jj6pg9mztv3ymxgkz80000gn/T/ipykernel_9508/3099769027.py:3: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats('retina')


In [4]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.linear_model import LinearRegression
import seaborn as sns
import warnings
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
import os
import json
from numpy.random import permutation
from sklearn import svm, datasets
from sacred import Experiment
from sacred.observers import FileStorageObserver
from IPython.display import set_matplotlib_formats

warnings.filterwarnings('ignore')

set_matplotlib_formats('retina')

PROJECT_ID='nyctaxi-demand-forecast'

In [5]:
# Notebooks must specify interactive=True, and this option is not required if it is in script format.
ex = Experiment('iris_rbf_svm', interactive=True)
ex.observers.append(FileStorageObserver.create('my_runs'))

# Save config as Decorator
@ex.config
def cfg():
    C = 1.0
    gamma = 0.7

# Specify ex.main => At this point, the factors in cfg are automatically injected.
# Also, in the Notebook, ex.Write main and use ex.automain in script file

@ex.main
def run(C, gamma):
    iris = datasets.load_iris()
    per = permutation(iris.target.size)
    iris.data = iris.data[per]
    iris.target = iris.target[per]
    clf = svm.SVC(C=C, kernel='rbf', gamma=gamma)
    clf.fit(iris.data[:90],
          iris.target[:90])
    return clf.score(iris.data[90:],
                   iris.target[90:])

In [6]:
run_result = ex.run()

INFO - iris_rbf_svm - Running command 'run'
INFO - iris_rbf_svm - Started run with ID "1"
INFO - iris_rbf_svm - Result: 0.9666666666666667
INFO - iris_rbf_svm - Completed after 0:00:00


In [7]:
run_result.config

{'C': 1.0, 'gamma': 0.7, 'seed': 715194634}

In [8]:
run_result.result

0.9666666666666667

In [9]:
run_result.experiment_info

{'name': 'iris_rbf_svm',
 'base_dir': '/Users/shim/ml-python-NYCtaxiDemandForecast/models',
 'sources': [],
 'dependencies': ['ipython==7.31.1',
  'ipywidgets==7.6.5',
  'matplotlib==3.7.1',
  'numpy==1.21.5',
  'pandas==1.5.1',
  'sacred==0.8.4',
  'scikit-learn==1.1.2'],
 'repositories': [],
 'mainfile': None}

### Integrate into existing projects

In [10]:
ex = Experiment('nyc-demand-prediction', interactive=True)

experiment_dir = os.path.join('./', 'experiments')
if not os.path.isdir(experiment_dir): 
    os.makedirs(experiment_dir)
ex.observers.append(FileStorageObserver.create(experiment_dir))

### Pre-Processing


In [11]:
%%time
base_query = """
WITH base_data AS 
(
  SELECT nyc_taxi.*, gis.* EXCEPT (zip_code_geom)
  FROM (
    SELECT *
    FROM `bigquery-public-data.new_york.tlc_yellow_trips_2015`
    WHERE 
        EXTRACT(MONTH from pickup_datetime) = 1
        and pickup_latitude  <= 90 and pickup_latitude >= -90
    ) AS nyc_taxi
  JOIN (
    SELECT zip_code, state_code, state_name, city, county, zip_code_geom
    FROM `bigquery-public-data.geo_us_boundaries.zip_codes`
    WHERE state_code='NY'
    ) AS gis 
  ON ST_CONTAINS(zip_code_geom, st_geogpoint(pickup_longitude, pickup_latitude))
)

SELECT 
    zip_code,
    DATETIME_TRUNC(pickup_datetime, hour) as pickup_hour,
    EXTRACT(MONTH FROM pickup_datetime) AS month,
    EXTRACT(DAY FROM pickup_datetime) AS day,
    CAST(format_datetime('%u', pickup_datetime) AS INT64) -1 AS weekday,
    EXTRACT(HOUR FROM pickup_datetime) AS hour,
    CASE WHEN CAST(FORMAT_DATETIME('%u', pickup_datetime) AS INT64) IN (6, 7) THEN 1 ELSE 0 END AS is_weekend,
    COUNT(*) AS cnt
FROM base_data 
GROUP BY zip_code, pickup_hour, month, day, weekday, hour, is_weekend
ORDER BY pickup_hour
"""

base_df = pd.read_gbq(query=base_query, dialect='standard', project_id=PROJECT_ID)

WARNING - google.auth.compute_engine._metadata - Compute Engine Metadata server unavailable on attempt 1 of 3. Reason: timed out
WARNING - google.auth.compute_engine._metadata - Compute Engine Metadata server unavailable on attempt 2 of 3. Reason: timed out
WARNING - google.auth.compute_engine._metadata - Compute Engine Metadata server unavailable on attempt 3 of 3. Reason: [Errno 64] Host is down
WARNING - google.auth._default - Authentication failed using Compute Engine authentication due to unavailable metadata server.
INFO - pandas_gbq.gbq - Total time taken 7.39 s.
Finished at 2023-03-21 12:29:41.


CPU times: user 1.38 s, sys: 164 ms, total: 1.55 s
Wall time: 14.9 s


### Feautre Engineering

In [12]:
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(base_df[['zip_code']])
ohe_output = enc.transform(base_df[['zip_code']]).toarray()
ohe_df = pd.concat([base_df, pd.DataFrame(ohe_output, columns='zip_code_'+enc.categories_[0])], axis=1)
ohe_df['log_cnt'] = np.log10(ohe_df['cnt'])

In [13]:
def split_train_and_test(df, date):
    """
    Dataframe divided by train_df and test_df

    df : Time series data frame
    date : Reference point date
    """
    train_df = df[df['pickup_hour'] < date]
    test_df = df[df['pickup_hour'] >= date]
    return train_df, test_df

### Split Train / Test 

In [14]:
train_df, test_df = split_train_and_test(ohe_df, '2015-01-24')

In [15]:
train_df.tail()

,zip_code,pickup_hour,month,day,weekday,hour,is_weekend,cnt,zip_code_10001,zip_code_10002,...,zip_code_12729,zip_code_12771,zip_code_13029,zip_code_13118,zip_code_13656,zip_code_13691,zip_code_14072,zip_code_14527,zip_code_14801,log_cnt
65113,11106,2015-01-23 23:00:00+00:00,1,23,4,23,0,34,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.531479
65114,10003,2015-01-23 23:00:00+00:00,1,23,4,23,0,2310,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.363612
65115,10454,2015-01-23 23:00:00+00:00,1,23,4,23,0,1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
65116,11435,2015-01-23 23:00:00+00:00,1,23,4,23,0,2,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.30103
65117,11208,2015-01-23 23:00:00+00:00,1,23,4,23,0,1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


- Delete columns I don't want to use

In [16]:
del train_df['zip_code']
del train_df['pickup_hour']
del test_df['zip_code']
del test_df['pickup_hour']

In [18]:
train_df.head(2)

,month,day,weekday,hour,is_weekend,cnt,zip_code_10001,zip_code_10002,zip_code_10003,zip_code_10004,...,zip_code_12729,zip_code_12771,zip_code_13029,zip_code_13118,zip_code_13656,zip_code_13691,zip_code_14072,zip_code_14527,zip_code_14801,log_cnt
0,1,1,3,0,0,169,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.227887
1,1,1,3,0,0,10,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [19]:
y_train_raw = train_df.pop('cnt')
y_train_log = train_df.pop('log_cnt')
y_test_raw = test_df.pop('cnt')
y_test_log = test_df.pop('log_cnt')

In [20]:
y_true = y_test_raw.values.copy()

In [26]:
x_train = train_df.copy()
x_test = test_df.copy()

In [27]:
def evaluation(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    score = pd.DataFrame([mape, mae, mse], index=['mape', 'mae', 'mse'], columns=['score']).T
    return score

### Experimental Settings
- Ex = Experimental ('nyc-demand-prediction', interactive=True') above, but save the setting as ex.config
- ex.capture uses the corresponding setting to return the function
- ex.main contains what to do when the experiment runs

In [28]:
@ex.config
def config():
    fit_intercept=True
    normalize=False

In [23]:
@ex.capture
def get_model(fit_intercept, normalize):
    return LinearRegression(fit_intercept=fit_intercept)

In [29]:
# _log and _run can be used as factors for functions without having to be defined separately
@ex.main
def run(_log, _run):
    lr_reg = get_model()
    lr_reg.fit(x_train, y_train_raw)
    pred = lr_reg.predict(x_test)
    # Save log to logFile
    _log.info("Predict End")
    score = evaluation(y_test_raw, pred)
    _run.log_scalar('model_name', lr_reg.__class__.__name__)
    
    # If you want to save it on Metrics side, use it as below.
    _run.log_scalar('metrics', score)
    
    # If you want to save it to the result side, use it as below.
    return score.to_dict()


In [30]:
experiment_result = ex.run()

INFO - nyc-demand-prediction - Running command 'run'
INFO - nyc-demand-prediction - Started run with ID "3"
INFO - run - Predict End
INFO - nyc-demand-prediction - Result: {'mape': {'score': 20309311547.545113}, 'mae': {'score': 207606380.14054915}, 'mse': {'score': 4.104279475456833e+19}}
INFO - nyc-demand-prediction - Completed after 0:00:04


In [31]:
experiment_result.config

{'fit_intercept': True, 'normalize': False, 'seed': 795848682}

### Parser to verify Experience
- The function to use depends on how you take log in Experiment
  - 1) If you store metrics in \_run.log\_scalar: recommended
  - 2) @ex.To return the result to the function of the main

In [32]:
# 1) If you store metrics in _run.log_scalar
def parsing_output(ex_id):
    with open(f'./experiments/{ex_id}/metrics.json') as json_file:
        json_data = json.load(json_file)
    with open(f'./experiments/{ex_id}/config.json') as config_file:
        config_data = json.load(config_file)
    
    output_df = pd.DataFrame(json_data['model_name']['values'], columns=['model_name'], index=['score'])
    output_df['experiment_num'] = ex_id
    output_df['config'] = str(config_data)
    metric_df = pd.DataFrame(json_data['metrics']['values'][0]['values'])
    
    output_df = pd.concat([output_df, metric_df], axis=1)
    output_df = output_df.round(2)
    return output_df

In [33]:
# 2) @ex.To return the result to the function of the main
def parsing_output(ex_id):
    with open(f'./experiments/{ex_id}/run.json') as json_file:
        json_data = json.load(json_file)
    output = pd.DataFrame(json_data['result'])
    return output

In [35]:
parsing_output(3)

,mae,mape,mse
score,2.076064e+08,2.030931e+10,4.104279e+19


### If you're curious about more details,
- [Sacred Github](https://github.com/IDSIA/sacred)
- [Introducing Python Sacred to help with the machine learning experiment] (https://zzsza.github.io/mlops/2019/07/21/python-sacred/))
- [Experimental and log monitoring using Sacred and Omniboard] (https://zzsza.github.io/mlops/2019/07/22/sacred-with-omniboard/)